## 0. Preparación del ambiente de python

In [1]:
# Instalacion de las dependencias
!pip install langchain langchain-openai langchain-chroma langchain-community openai chromadb PyPDF2 pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.0 MB/s eta

In [2]:
# Paso 0: Importamos las librerias
import os
#from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI

In [3]:
#Cargamos el TOKEN de OpenAI para poder ejecutar prompts
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Extración los archivos TXT o PDF

In [5]:
# Paso 1: Función para cargar documentos
# Dependiendo del tipo de archivo (pdf o txt) llama la función requerida
def load_documents(file_paths):
    documents = []
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif file_path.endswith('.txt'):
            loader = TextLoader(file_path)
        else:
            continue
        documents.extend(loader.load())
    return documents

## 2. División de los textos en fragmentos

In [6]:
# Paso 2: Función para dividir texto en fragmentos
# Divide el texto en chunks de tamaño 1000 con un overlap de 200 para mantener
# la continuidad de la información
def chunk_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_documents(documents)

## 3. Creación de los embeddings

In [ ]:
# Paso 3: Función para crear embeddings
# Creamos los embeddings de los chunks directamente cuando se llama a la funcion
# de almacenamiento de los vectores
# Se crea la funcion en el cuerpo de la funcion main

## 4. Almacenamiento de los vectores (embeddings) en la base de datos

In [7]:
# Paso 4: Almacenamos los emebeddings en la bases de datos de Chroma
# Si la base de datos no existe, la crea; si existe, solamente la carga
# Esto para evitar que se tenga que generar desde 0 la base de datos,
# cada vez que se reciba un prompt del usuario
def store_in_chromadb(chunks, embeddings_model):
    report = 'RetoRAG'
    db_path = "/content/drive/My Drive/TecdeMonterrey/chroma_db"
    if not os.path.exists(db_path):
        # Create the vector store and persist it
        vector_store = Chroma.from_documents(chunks, embeddings_model, collection_name=report,persist_directory=db_path)
    else:
        # Load the existing vector store
        vector_store = Chroma(persist_directory=db_path, embedding_function=embeddings_model,collection_name=report)
    return vector_store

## 5. Retriving from the Persistant Vector Datastore

In [8]:
# Paso 5: Recuperar información relevante
# Obtenemos los 5 primeros fragmentos relevantes de la base de datos
def retrieve_relevant_info(question, vector_store):
    docs = vector_store.similarity_search(question, k=5)
    return docs

## 6. Generar prompt usando los vectores relevantes y el prompt del usuario

In [9]:
# Paso 6: Generar prompt
# Generamos el prompt usando el prompt del usuario, mas los
# fragmentos relevantes
def generate_prompt(question, relevant_docs):
    limit=3750
    # Extraemos el contexto de los documentos
    contexts = [doc.page_content for doc in relevant_docs]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {question}\nAnswer:"
    )

    # Initialize prompt with all contexts
    prompt = (
        prompt_start +
        "\n\n---\n\n".join(contexts) +
        prompt_end
    )

    # If total length exceeds limit, reduce contexts one by one
    for i in range(len(contexts)-1, 0, -1):
        if len("\n\n---\n\n".join(contexts[:i])) < limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i]) +
                prompt_end
            )
            break

    return prompt

## 7. Generar la respuesta usando GPT-4

In [10]:
# Paso 7: Generar respuesta usando GPT-4
# Enviamos el prompt al modelo

def generate_answer(prompt):
    llm = ChatOpenAI(model_name="gpt-4", api_key=api_key, temperature=0)
    return llm.invoke(prompt)

## 8. Funcion principal

In [11]:
# Función principal
#
def main(file_paths, question):
    # Cargamos los documentos
    documents = load_documents(file_paths)
    # Fragmentamos los documentos
    chunks = chunk_documents(documents)
    # Definimos la función que realizara los embeddings
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small",api_key=api_key)
    # Creamos o guardamos la base de datos de vactores
    vector_store = store_in_chromadb(chunks,embeddings)
    # Busacamos los vectores mas relevantes
    relevant_docs = retrieve_relevant_info(question, vector_store)
    # Generamos el prompt usando la pregunta y el contexto obetenido
    prompt = generate_prompt(question, relevant_docs)
    # Obtenemos la respuesta del LLM
    answer = generate_answer(prompt)
    return answer

## 9. Programa Principal

In [ ]:
# Función principal
if __name__ == "__main__":
    file_paths = ['/content/drive/MyDrive/TecdeMonterrey/Generacion de Codigo con  AI generativa/Reto1/V-GEL Información ES PDF.pdf', '/content/drive/MyDrive/TecdeMonterrey/Generacion de Codigo con  AI generativa/Reto1/RETO_RAG.pdf']
    question = "¿Cuál es el objetivo del proyecto?"
    answer = main(file_paths, question)
    print(answer)

## 10. Preguntas al sistema RAG

In [12]:
# Primera pregunta
file_paths = ['/content/drive/MyDrive/TecdeMonterrey/Generacion de Codigo con  AI generativa/Reto1/V-GEL Información ES PDF.pdf', '/content/drive/MyDrive/TecdeMonterrey/Generacion de Codigo con  AI generativa/Reto1/RETO_RAG.pdf']
question = "¿Cuáles son las instrucciones para el alumno en la Actividad: Creación de un sistema RAG sencillo?"
answer = main(file_paths, question)
print(f"Pregunta : {question}. \n Respuesta {answer.content}")

Pregunta : ¿Cuáles son las instrucciones para el alumno en la Actividad: Creación de un sistema RAG sencillo?. 
 Respuesta Las instrucciones para el alumno en la Actividad: Creación de un sistema RAG sencillo son las siguientes:

1. Leer el caso: El alumno es parte del equipo encargado de prototipar una solución RAG que permita responder preguntas sobre un conjunto de documentos internos.

2. Objetivo: En 40 minutos, el alumno deberá crear un prototipo funcional que permita cargar o indexar al menos 2 documentos, reciba una pregunta del usuario y utilice un modelo generativo para responder la pregunta, apoyándose en la información recuperada de los documentos.

3. Herramientas sugeridas: Se sugiere el uso de Python, LangChain, ChromaDB o similares. El alumno puede usar ejemplos y plantillas de la documentación oficial.

4. Entrega: Al final, el alumno debe compartir el código fuente y una breve explicación de cómo lo implementó. Debe usar técnicas de prompt engineering para definir la 

In [14]:
# Segunda Pregunta
question = "¿Cuáles es el contexto del caso en la Actividad: Creación de un sistema RAG sencillo?"
answer = main(file_paths, question)
print(f"Pregunta : {question}. Respuesta: {answer.content}")

Pregunta : ¿Cuáles es el contexto del caso en la Actividad: Creación de un sistema RAG sencillo?. Respuesta: El contexto del caso en la Actividad: Creación de un sistema RAG sencillo es que una pequeña empresa necesita implementar un sistema que permita a sus empleados consultar información interna de manera eficiente. El equipo de desarrollo ha decidido utilizar un enfoque RAG (Retrieval-Augmented Generation) apoyado en modelos generativos, para responder preguntas sobre documentos internos como manuales, políticas o reportes. El objetivo es crear un prototipo funcional que permita cargar o indexar al menos 2 documentos, reciba una pregunta del usuario y utilice un modelo generativo para responder la pregunta, basándose en la información recuperada de los documentos.


In [15]:
# Tercera pregunta
question = "¿Cuáles sugerencias de pasos en la Actividad: Creación de un sistema RAG sencillo?"
answer = main(file_paths, question)
print(f"Pregunta : {question}. Respuesta: {answer.content}")

Pregunta : ¿Cuáles sugerencias de pasos en la Actividad: Creación de un sistema RAG sencillo?. Respuesta: Las sugerencias de pasos en la Actividad: Creación de un sistema RAG sencillo son las siguientes:

1. Usa técnicas de prompt engineering para definir la arquitectura del sistema RAG.
2. Instala las librerías necesarias.
3. Indexa los documentos usando una base de datos vectorial (como ChromaDB).
4. Implementa una función que reciba una pregunta, recupere los fragmentos más relevantes y los pase al modelo generativo.
5. Prueba tu sistema con al menos dos preguntas distintas.


In [16]:
# Tercera pregunta
question = "¿Cuáles son las ventajas de v-gel?"
answer = main(file_paths, question)
print(f"Pregunta : {question}. Respuesta: {answer.content}")

Pregunta : ¿Cuáles son las ventajas de v-gel?. Respuesta: Las ventajas de v-gel incluyen un mejor diseño gracias a años de investigación y comentarios de los clientes, es de un solo uso siguiendo las normas de higiene de la medicina humana. Los dispositivos v-gel son fáciles y rápidos de colocar, útiles en emergencias de resucitación y adecuados para una amplia variedad de procedimientos, incluyendo trabajo dental. Además, evitan traumatismos laríngeos y traqueales, lo que significa un procedimiento anestésico más seguro y una cómoda recuperación del paciente. También evitan la tos y las náuseas postoperatorias, tienen una baja resistencia de la vía aérea gracias al gran canal de aire dentro del dispositivo, y proporcionan un sellado de alta calidad que evita fugas de agentes anestésicos volátiles. Finalmente, ayudan a prevenir y controlar la regurgitación del estómago y son ideales para un procedimiento dental seguro.


In [17]:
# Cuarta pregunta
question = "¿Qué es v-gel?"
answer = main(file_paths, question)
print(f"Pregunta : {question}. Respuesta: {answer.content}")

Pregunta : ¿Qué es v-gel?. Respuesta: V-GEL es el primer dispositivo supraglótico para vías respiratorias de uso veterinario, utilizado para suministrar gases anestésicos y/u oxígeno a los pacientes, especialmente gatos y conejos. Está diseñado para entrar en la faringe y evitar problemas causados por traumatismos laríngeos y traqueales. Es fácil y rápido de colocar y útil en emergencias de resucitación. También es adecuado para una amplia variedad de procedimientos, incluido el trabajo dental.


In [18]:
# Quinta pregunta
question = "¿Cuáles son las caracteristicas clave del dispositivo v-gel advanced?"
answer = main(file_paths, question)
print(f"Pregunta : {question}. Respuesta: {answer.content}")

Pregunta : ¿Cuáles son las caracteristicas clave del dispositivo v-gel advanced?. Respuesta: El dispositivo v-gel Advanced tiene un mejor diseño gracias a años de investigación y comentarios de los clientes. Es de un solo uso, siguiendo las normas de higiene de la medicina humana. Conserva las características originales de los dispositivos v-gel para gato y conejo, incluyendo funciones de coincidencia anatómica combinadas con un material suave para brindar un sellado de alta calidad. Es adecuado para pacientes conectados a un ventilador mecánico y evita traumatismos laríngeos y traqueales. Permite una colocación rápida, fácil, segura y sin estrés. No causa tos ni náuseas postoperatorias y tiene una baja resistencia de la vía aérea gracias al gran canal de aire dentro del dispositivo. Ofrece un sellado de alta calidad que evita fugas de agentes anestésicos volátiles, cuidando la seguridad del personal. Facilita una recuperación más segura y cómoda. Tiene un inserto de punta en la parte 